In [2]:
import pandas as pd
import re

In [3]:
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx', index_col='park_id')
data.head(7)

,park_name,attractions,area,position,district_id,rating
park_id,,,,,,
0,Park-Arkadia,"stawy parkowe, muzeum im. Xawerego Dunikowskie...",11,"pomiędzy ulicami Puławską, Żywnego, Piaseczyńs...",0,NaN
1,Park-im-Marszalka-Edwarda-Rydza-Smiglego,"fontanna i układ wodny od strony ul .Rozbrat, ...",53,"w rejonie ulic Czerniakowskiej, Górnośląskiej,...",8,NaN
2,Park-Kazimierzowski,"stary drzewostan, piękny widok rozciągający si...",4,"w obrębie ulic Karowej, Browarnej, Dynasy",8,NaN
3,Park-Skaryszewski-im-I-J-Paderewskiego,"stary drzewostan, układ wodny, grota, plac zab...","49,56","w obrębie ulic: Al. Waszyngtona, Zielenieckiej...",1,NaN
4,Park-Znicza,"plac zabaw, miejsce pamięci, fontanna","1,9","w obrębie ulic Znicza, Filomatów, Bełżecka, Ml...",1,NaN
5,Park-Zeromskiego,"stary drzewostan, plac zabaw, fontanna z rzeźb...",6,"przy pl. Wilsona, pomiędzy ulicami Mickiewicza...",13,NaN
6,Zieleniec-Wielkopolski,"plac zabaw, duża różnorodność gatunkowa roślin...","4,95","w obrębie ulic Filtrowej, Wawelskiej, Łęczycki...",10,NaN


In [336]:
# Formating of area to digit format
area = data['area']
pattern = re.compile(r'\d+,*\d*')
digit = area.str.findall(pattern)
number = digit.apply(lambda lst: lst[0])

In [ ]:
# the table of Warsaw district.
html_tables = pd.read_html(r'https://pl.wikipedia.org/wiki/Podzia%C5%82_administracyjny_Warszawy')
districts = html_tables[0]['Dzielnica']
districts.index = pd.RangeIndex(len(districts), name='district_id')
districts = districts.str.replace('-', ' ')

In [ ]:
# districts in park table
pattern = re.compile('|'.join(districts))
p_districts = data['position']
p_districts = p_districts.str.findall(pattern).apply(lambda lst: lst[0] if len(lst) >0 else None)
p_districts

In [ ]:
# districts for the park table.
districts_index = pd.Index(districts)
district_id = districts_index.get_indexer(p_districts)

In [352]:
# streets address of parks. 
pattern = re.compile(r',',re.UNICODE)
buffer = data['position'].str.split(pattern, n=1, expand=True)
buffer = buffer.ffill(axis=1).iloc[:, 1]
position = buffer.apply(lambda s: s.strip())

In [368]:
# writing the data to df.
data['area'] = number
data['district_id'] = district_id
data['position'] = position

In [ ]:
# To write the data to excel
# with pd.ExcelWriter(r'D:\WarsawParkInfo\data\table.xlsx', mode='w') as writer:
#     data.to_excel(writer, sheet_name='Park', index=False)
#     districts.to_excel(writer, sheet_name='district', index=True)

In [4]:
# Creating of district of park table.
# key: Mokotow: 0, Ochota: 10, Śródmieście: 8.
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx')
p_district_id = data[['district_id', 'park_id']]

# drop the record of park Pole-Mokotowskie
head, tail = p_district_id[:55], p_district_id[56:]
new_records = pd.DataFrame({'park_id': [55, 55, 55], 'district_id': [0, 10, 8]})
districts_of_park = pd.concat([head, tail, new_records])

with pd.ExcelWriter(r'D:\WarsawParkInfo\data\buffer.xlsx') as writer:
    districts_of_park.to_excel(writer, sheet_name='Districts_of_park', index=False) 